In [1]:
!pip install plotly

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import re
from geopy.geocoders import Nominatim
import json
import requests


REST_API_KEY = "24e7619a54b8e426d7f7e06a44973320"
data = pd.read_csv("대전광역시_온통대전 신청 가맹점 목록_20211130.csv",encoding = "EUC-KR")
jsonfile = open("동구.json", 'r', encoding = "UTF-8").read()
jsondata = json.loads(jsonfile)

def find_coord_kakao(address):
    def find_coord_geo(address):
        try:
            geolocoder = Nominatim(user_agent="South Korea", timeout=None)
            geo = geolocoder.geocode(address)

            return geo.longitude, geo.latitude
        except:
            return 0, 0
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json?query={address}".format(address = address)
        headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        coord = result["documents"][0]["address"]
        return float(coord["x"]), float(coord["y"])
    except:
        return find_coord_geo(address)
    # 카카오 api로 좌표를 찾고 없으면 geolocoder로 찾고 그래도 안되면 0,0 반환(경도-위도순)
    # 온통대전에서 동구인 것만 찾아 data에 넣음

data = data[data['구'] == '동구']
data["경도"] = ""
data["위도"] = ""
# 경도 위도를 

data['주소'] = data['주소'].str.split(',').str.get(0)
data['주소'] = data['주소'].str.split('(').str.get(0)
data.reset_index(inplace = True)  ## 인덱스 리셋
data = data.drop(["구", "순번", "index"], axis = "columns")
# 구, 순번, index 지움

for i in data.index:
    data.loc[i, "경도"] = find_coord_kakao(data.loc[i, "주소"])[0]
    data.loc[i, "위도"] = find_coord_kakao(data.loc[i, "주소"])[1]
    print(i)
data = data.drop(data[data["경도"] == 0].index)

# 입력받은 도로명 주소를 좌표로 변환해서 그를 중심좌표로 맵을 생성
home_address = input("도로명 주소를 입력하세요")
home_coord = find_coord_kakao(home_address)

# foium은 위도 경도 순으로 받아서 
m = folium.Map(location = [home_coord[1], home_coord[0]], zoom_start = 11)

# cluster를 모을 것을 만듦(마커들의 집합을 만듦)
marker_cluster = MarkerCluster().add_to(m)


# 마커 클러스트라는 집합 안에 좌표로 찍은 마커를 넣음
for i in data.index:
  folium.Marker(location = [data.loc[i, "위도"], data.loc[i, "경도"]], popup = data.loc[i, "가맹점"], icon = folium.Icon(color = "blue", icon = "star"),
  ).add_to(marker_cluster)

# 동을 기준으로 그룹화하여 가맹점 수를 카운트
data = pd.DataFrame(data.groupby("동")["가맹점"].count())

# 인덱스 리셋(데이터프레임을 갱신할 때 마다 해주면 좋다) 그래야 loc 쓸 때 편함
data.reset_index(inplace = True)

# 열 이름을 EMD_NM로 바꿈
data = data.rename(columns = {"동" : "EMD_NM"})

# 가맹점수를 일반 int형으로 바꾸는 과정
for i in data.index:
    data.loc[i, "가맹점"] = str(data.loc[i, "가맹점"])

for i in data.index:
    data.loc[i, "가맹점"] = int(data.loc[i, "가맹점"])

# 폴리움 
folium.Choropleth(geo_data = jsondata,
                  data = data,
                  columns = ["EMD_NM", "가맹점"],
                  fill_color='YlGn',
                  key_on = 'feature.properties.EMD_NM').add_to(m)  # columns = ["EMD_NM" <-와 key_on = 'feature.properties.EMD_NM' <-의 타입이 맞아양 함
# Choropleth 왼쪽 인자들은 필수로 있어야 함(기본 형식) 
m.save("qwer.html")  # 저장해서 웹브라우저로 읽기

NameError: name 'find_coord_kakao' is not defined